In [1]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [5]'.</span>

In [2]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [3]'.</span>

## FDA inspection data analytics - build the dataset

## load libraries

In [3]:
file = '../data/input/raw_clinical_investigator_inspection_fc.xls'
file2 = '../data/input/raw_clinical_investigator_inspection_da.xls'
file_map = '../data/input/refined_mapping_fda_inspections_code.csv'
file_out = '../data/output/final_set_FDA.csv'

In [4]:
# Parameters
file = "data/input/raw_clinical_investigator_inspection_fc.xls"
file2 = "data/input/raw_clinical_investigator_inspection_da.xls"
file_map = "data/input/refined_mapping_fda_inspections_code.csv"
file_out = "data/output/final_set_FDA.csv"


In [5]:
import numpy as np
import os as os
import pandas as pd
import xlrd 


## load and clean fda inspections data

In [6]:
xl = pd.ExcelFile(file)

df_fc = xl.parse('Sheet1')


xl = pd.ExcelFile(file2)

df_da = xl.parse('Sheet1')

In [7]:
print(df_da.shape)
print (df_fc.shape)

(2500, 12)
(1748, 12)


In [8]:
# append the two df
df_insp = df_da.append(df_fc, ignore_index = True)

In [9]:
print(df_insp.shape)

(4248, 12)


In [10]:
df_insp

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,Insp. Class.,Insp. Def. Codes
0,953,"Iber, Frank L MD",Edward Hines Jr Hosp,Po Box 5000,Hines,IL,USA,60141,,DA,WASH,
1,1774,"Jolly, Henry W JR MD",Louisiana State Univ,1542 Tulane Ave,New Orleans,LA,USA,70112,,DA,WASH,
2,2211,"Y, Joseph R MD",Ohio State Univ,333 W 10Th Ave,Columbus,OH,USA,43210,,DA,WASH,
3,5878,"Pine, Michael B MD",Va Med Ctr,-,Long Beach,CA,USA,90822,,DA,CANC,
4,6280,"Sung, Ruey J MD",Stanford Univ Med Ctr,300 Pasteur Dr Rm H2146,Stanford,CA,USA,94305,,DA,CANC,
...,...,...,...,...,...,...,...,...,...,...,...,...
4243,707559,"Raghavan, Preeti MD","Rusk Rehabilitation, New York Univ.School Of M...","240 East 38Th Street, Ambulatory Care",New York,NY,USA,10016,10/11/18,FC,NAI,00
4244,710371,"Phan, Han C MD","Rare Disease Research, Llc",1891 Howell Mill Rd Nw Ste B,Atlanta,GA,USA,30318,11/15/18,FC,NAI,00
4245,712118,"Aslam, Zaheer",Gulfcoast Clinical Research Center,6150 Diamond Centre Court Bldg 500,Fort Myers,FL,USA,33912,12/10/18,FC,NAI,00
4246,713788,"Ledo-Sanchez, Gustavo G","Best Quality Research, Inc",2387 West 68Th Street Suite 403,Hialeah,FL,USA,33016,10/29/18,FC,NAI,00


In [11]:
# rename the 'Insp. Def. codes' to 'insp_codes' and 
df_insp.rename(columns={'Insp. Def. Codes':'insp_codes'}, inplace=True)
df_insp.rename(columns={'Insp. Class.':'insp_class'}, inplace=True)

In [12]:
# convert code '00' and '0' (=no finding noted) into '22' to avoid issues mixing up with NaN
df_insp['insp_codes'].replace(['00'], '22', inplace=True)
df_insp['insp_codes'].replace(['0'], '22', inplace=True)

In [13]:
# convert empty/blank cells in 'insp_codes'(=with no inspection data) into NaN
df_insp = df_insp.replace(r'^\s+$', np.nan, regex=True)

In [14]:
# drop washout inspections
df_insp1 = df_insp[df_insp.insp_class != 'WASH']
print(df_insp1.shape)

(4199, 12)


In [15]:
# drop canceled inspections
insp_data = df_insp1[df_insp1.insp_class != 'CANC']
print(insp_data.shape)

(4072, 12)


In [16]:
# look for inspections with no action required 'NAI' but w/o code assigned

insp_data_NULL = insp_data[insp_data['insp_codes'].isnull()]
print(insp_data_NULL.shape)

(961, 12)


In [17]:
insp_data_NAI_and_null = insp_data_NULL[insp_data_NULL['insp_class'] == 'NAI']
print(insp_data_NAI_and_null.shape)

(639, 12)


In [18]:
# there are inspections with insp_class = No Action Indicated (NAI) w/o inspection code
# assign them 'insp_code' = 22

insp_data.loc[(insp_data['insp_class'] == 'NAI'), 'insp_codes'] = '22'
print(insp_data.shape)

(4072, 12)


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
# check numbers add up

print((insp_data[insp_data['insp_codes'].isnull()]).shape)

# 961 rows with NaN = 639 insp_class = 'NAI' AND insp_code= '22' + 322 insp_code = NaN
# ---> OK

(322, 12)


In [20]:
# look for rows with typos
value_list = [',', ',,',',,,']

In [21]:
insp_data[insp_data.insp_codes.isin(value_list)]

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
311,56699,"Scharf, Martin B PHD",Ctr Res Sleep Disorders/Tristate Sleep Disorde...,1275 E Kemper Rd,Cincinnati,OH,USA,45246,02/06/01,DA,OAIW,","
2783,99809,"Moskowitz, George MD",-,101 Rutledge St,Charlestown,SC,USA,00000,05/29/81,FC,OAIC,","


In [22]:
# replace typo with '23' (=no data available)
insp_data['insp_codes'].replace([','], '23', inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [23]:
# replace typo (,00) - replace by 22
insp_data['insp_codes'].replace([',00'], '22', inplace=True)

In [24]:
insp_data

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
43,449958,"Wiley, Kenneth L MD",New Orleans Clinl Trials Management,2200 Veterans Mem Blvd Ste 209,Kenner,LA,USA,70062,NaN,DA,MTF,NaN
46,539971,"Benzar, Zinoviy MD",Brooklyn Med Inst,2072 Bath Ave,Brooklyn,NY,USA,11214,NaN,DA,MTF,NaN
47,13321,"Nathan, Robert A MD","Asthma & Allergy Associates, P.C.",2709 N Tejon Std Ste. 202,Colorado Sprgs,CO,USA,80907,01/01/95,DA,VAI,"03,05,06"
48,438326,"Young, James E MD",Michigan Reproductive & Ivf Ctr,221 Michigan Ste 406,Grand Rapids,MI,USA,49503,01/02/01,DA,VAI,"03,04,05,06"
49,228659,"Peterson, Daniel L MD",Sierra Internal Medicine Assoc,865 N Tahoe Blvd Ste 306 Box 7870,Incline Village,NV,USA,89451,01/02/13,DA,VAI,05
...,...,...,...,...,...,...,...,...,...,...,...,...
4243,707559,"Raghavan, Preeti MD","Rusk Rehabilitation, New York Univ.School Of M...","240 East 38Th Street, Ambulatory Care",New York,NY,USA,10016,10/11/18,FC,NAI,22
4244,710371,"Phan, Han C MD","Rare Disease Research, Llc",1891 Howell Mill Rd Nw Ste B,Atlanta,GA,USA,30318,11/15/18,FC,NAI,22
4245,712118,"Aslam, Zaheer",Gulfcoast Clinical Research Center,6150 Diamond Centre Court Bldg 500,Fort Myers,FL,USA,33912,12/10/18,FC,NAI,22
4246,713788,"Ledo-Sanchez, Gustavo G","Best Quality Research, Inc",2387 West 68Th Street Suite 403,Hialeah,FL,USA,33016,10/29/18,FC,NAI,22


In [25]:
# explose insp_codes column (requires pandas >0.25)
insp_data_interim = insp_data.assign(insp_codes=df_insp.insp_codes.str.split(',')).explode('insp_codes')

In [26]:
# check
insp_data_interim

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
43,449958,"Wiley, Kenneth L MD",New Orleans Clinl Trials Management,2200 Veterans Mem Blvd Ste 209,Kenner,LA,USA,70062,NaN,DA,MTF,NaN
46,539971,"Benzar, Zinoviy MD",Brooklyn Med Inst,2072 Bath Ave,Brooklyn,NY,USA,11214,NaN,DA,MTF,NaN
47,13321,"Nathan, Robert A MD","Asthma & Allergy Associates, P.C.",2709 N Tejon Std Ste. 202,Colorado Sprgs,CO,USA,80907,01/01/95,DA,VAI,03
47,13321,"Nathan, Robert A MD","Asthma & Allergy Associates, P.C.",2709 N Tejon Std Ste. 202,Colorado Sprgs,CO,USA,80907,01/01/95,DA,VAI,05
47,13321,"Nathan, Robert A MD","Asthma & Allergy Associates, P.C.",2709 N Tejon Std Ste. 202,Colorado Sprgs,CO,USA,80907,01/01/95,DA,VAI,06
...,...,...,...,...,...,...,...,...,...,...,...,...
4243,707559,"Raghavan, Preeti MD","Rusk Rehabilitation, New York Univ.School Of M...","240 East 38Th Street, Ambulatory Care",New York,NY,USA,10016,10/11/18,FC,NAI,22
4244,710371,"Phan, Han C MD","Rare Disease Research, Llc",1891 Howell Mill Rd Nw Ste B,Atlanta,GA,USA,30318,11/15/18,FC,NAI,22
4245,712118,"Aslam, Zaheer",Gulfcoast Clinical Research Center,6150 Diamond Centre Court Bldg 500,Fort Myers,FL,USA,33912,12/10/18,FC,NAI,22
4246,713788,"Ledo-Sanchez, Gustavo G","Best Quality Research, Inc",2387 West 68Th Street Suite 403,Hialeah,FL,USA,33016,10/29/18,FC,NAI,22


In [27]:
# convert insp_code into numeric data for the join
insp_data_interim['insp_codes'] = pd.to_numeric(insp_data_interim['insp_codes'])

In [28]:
# Verify values in insp_codes
print(insp_data_interim.insp_codes.unique())

[nan  3.  5.  6.  4. 22. 16.  2. 12. 18.  1.  7. 15. 14.  0.  9.  8. 10.
 20. 19. 17. 11. 13.]


In [29]:
# check 
print(insp_data_interim[insp_data_interim['insp_codes'].isnull()])

      ID No.                              Name  \
43    449958        Wiley,   Kenneth   L    MD   
46    539971        Benzar,   Zinoviy       MD   
54    177764          Kerr,   Robert   O    MD   
59     82644                Schmunes,   Edward   
62    195131          Ghali,   Jalal   K    MD   
...      ...                               ...   
4159  642028                Chisti,   Mohammad   
4165  648641                   Driver,   Brian   
4181  661444  Castellon,   Celestino   P    MD   
4187  666448                  Racke,   Michael   
4235  697186                Meltzer,   Herbert   

                                               Location  \
43                  New Orleans Clinl Trials Management   
46                                    Brooklyn Med Inst   
54                                        Sw Reg Ca Ctr   
59                                                    -   
62                         Louisiana State Univ Med Ctr   
...                                          

In [30]:
#find duplicates

duplicateRowsDF = insp_data_interim[insp_data_interim.duplicated()]

print(duplicateRowsDF)


      ID No.                          Name  \
238   318343    Caruso,   Walter   A    MD   
297   426812    Duckor,   Steven   L    MD   
311    56699   Scharf,   Martin   B    PHD   
683   142756    Gammon,   Walter   R    MD   
860   574154     Cannon,   Kevin   D    MD   
...      ...                           ...   
3870  499117  Montero,   Alberto   J    MD   
4036  542883      Renfroe,   Ben   J    MD   
4194  669694   Huffman,   Cynthia       MD   
4222  683889             Lang,   Jason   E   
4223  684207                Amin,   Faisal   

                                               Location  \
238                                                   -   
297                               Agmg Endoscopy Center   
311   Ctr Res Sleep Disorders/Tristate Sleep Disorde...   
683                                    Nc Memorial Hosp   
860                      Pmg Research Of Wilmington Llc   
...                                                 ...   
3870                            Mu

In [31]:
# drop duplicates
insp_data_final = insp_data_interim.drop_duplicates(keep='first')

In [32]:
# confirm # of rows with NaN
print((insp_data_final[insp_data_final['insp_codes'].isnull()]).shape)

(1035, 12)


## load mapping data 

In [33]:
# load csv file - created and refined based on:
# https://www.fda.gov/drugs/enforcement-activities-fda/clinical-investigator-inspection-list-ciil-database-codes)
mapping = pd.read_csv (file_map, encoding = 'cp1250',low_memory=False)

In [34]:
# check
mapping

,code,deficiency,cfr_reference,mapped_clinical_impact_factor
0,1,Records availability,21 CFR 312.62,data_integrity
1,2,Failure to obtain and/or document subject consent,"21 CFR 312.60, 50.20, 50.27",consent
2,3,Inadequate informed consent form,21 CFR 50.25,consent
3,4,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety
4,5,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety
5,6,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints
6,7,Unapproved concomitant therapy,21 CFR 312.60,primary_endpoints_and_safety
7,8,Inappropriate payment to volunteers,21 CFR 50.20,consent
8,9,Unapproved use of drug before IND submission,21 CFR 312.40(d),safety
9,10,Inappropriate delegation of authority,"21 CFR 312.7, 312.61",primary_endpoints_and_safety


In [35]:
# rename column to insp_codes to allow mapping with FDA dataset
mapping.rename(columns={'code':'insp_codes'}, inplace=True)

In [36]:
# insp_codes to numeric value
mapping['insp_codes'] = pd.to_numeric(mapping['insp_codes'])

## join and clean consolidated data set ('fda_inspections')

In [37]:
# do the join
finalset = pd.merge (insp_data_final,mapping, on='insp_codes', how='left')

In [38]:
# check
print(finalset.shape)
finalset.info()

(7132, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7132 entries, 0 to 7131
Data columns (total 15 columns):
ID No.                           7132 non-null int64
Name                             7132 non-null object
Location                         7122 non-null object
Address                          7132 non-null object
City                             7132 non-null object
State                            7132 non-null object
Cntry                            7132 non-null object
Zip                              7132 non-null object
Insp. Date                       7113 non-null object
Insp. Type                       7132 non-null object
insp_class                       7132 non-null object
insp_codes                       6097 non-null float64
deficiency                       6075 non-null object
cfr_reference                    5097 non-null object
mapped_clinical_impact_factor    6075 non-null object
dtypes: float64(1), int64(1), object(13)
memory usage: 891.5+ KB


## final cleaning

In [39]:
print(finalset.insp_codes.unique())

[nan  3.  5.  6.  4. 22. 16.  2. 12. 18.  1.  7. 15. 14.  0.  9.  8. 10.
 20. 19. 17. 11. 13.]


In [40]:
print(finalset[finalset['insp_codes'].isnull()])

      ID No.                              Name  \
0     449958        Wiley,   Kenneth   L    MD   
1     539971        Benzar,   Zinoviy       MD   
15    177764          Kerr,   Robert   O    MD   
23     82644                Schmunes,   Edward   
27    195131          Ghali,   Jalal   K    MD   
...      ...                               ...   
7020  642028                Chisti,   Mohammad   
7027  648641                   Driver,   Brian   
7047  661444  Castellon,   Celestino   P    MD   
7054  666448                  Racke,   Michael   
7116  697186                Meltzer,   Herbert   

                                               Location  \
0                   New Orleans Clinl Trials Management   
1                                     Brooklyn Med Inst   
15                                        Sw Reg Ca Ctr   
23                                                    -   
27                         Louisiana State Univ Med Ctr   
...                                          

In [41]:
# Add insp_code '23' --> "no data reported"

finalset['insp_codes'].fillna(23, inplace=True)

In [42]:
# confirm no NaN
print((finalset[finalset['insp_codes'].isnull()]).shape)

(0, 15)

In [43]:
# Export to csv the final set
finalset.to_csv(file_out)